Das Programm ist für das Laden von bereits berechneten Modellen und das Berechnen der Prediction für einen Satz von Testbildern (die nicht Bestandteil der Trainings und Validations Sets sind)

Die Quelle für die Kommentare ist Google Gemini

In [1]:
# Die benötigten imports, damit python die benutzen kann
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
import numpy as np


In [2]:
# mit google drive verbinden
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Die Variable saveModelFileName, welche den Dateinamen zu dem Modell hat, die anderen sind auskommentiert, da diese gerade nicht gebraucht werden
saveModelFileName = "/content/drive/Shareddrives/Models/epochen1000_128fil_64img_rgb.keras"

#saveModelFileName = "/content/drive/Shareddrives/Models/epochen1000_128fil_64img_gray.keras"

#saveModelFileName = "/content/drive/Shareddrives/Models/epochen8000_128fil_64img_gray.keras"

# die Image size muss immer angepasst werden je nach Modell, wenn das Modell eine andere Bildgrösse als 64x64 benutzt
IMG_SIZE = 64
image_size = (IMG_SIZE, IMG_SIZE)

# der usedColorMode ist entweder rgb oder grayscale
#usedColorMode = "grayscale"
usedColorMode = "rgb"

# Der Pfad/Verzeichnis von den Bildern, die zum Test genutzt werden. Dies sind andere Bilder als die, die zum Training genutzt wurden.
testDir = "/content/drive/Shareddrives/Testbilder/Test_Puzzle/"

model = keras.models.load_model(saveModelFileName)
# Model Summary wird zur Sicherheit nochmal ausgegeben.
model.summary()



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 64, 64, 128)         │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 131072)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       8,388,672 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 18)                  │           1,170 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,623,032 (97.74 MB)

 Trainable params: 8,541,010 (32.58 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 17,082,022 (65.16 MB)

In [5]:
# Klassennamen für die Zuordnung der verschiedenen Klassen zu Zahlen 0...17 (18 Klassen) für die spätere Verarbeitung in Tabellenform
classnames = ['010', '020', '030', '040', '050', '060', '070', '071', '080', '081', '090', '091', '100', '110', '120', '130', '140', '150']

# Ordner mit den Testbildern einlesen
testFiles = os.listdir(testDir)

# Anzahl Klassen ist 18, die 3 mittleren Bilder des Puzzles gibt es in 2 Ausrichtungen ('070', '071', '080', '081', '090', '091') und deshalb gibt es insgesamt 15 + 3 Klassen
# Numpy array anlegen mit den Dimensionen 19 x 19 für die 18 Klassen und 1 Zeile und 1 Spalte für die Zeilen und Spaltensummen
F1arr = np.zeros(shape=(19, 19))

# Durchgehen aller Dateien im Ordner testdir, der weiter oben ausgewählt wird
for curFile in testFiles:
      # hier wird der Bilderpath (testDir) mit dem Dateinamen des gerade bearbeiteten Bild zusammengesetzt, dies führt zu dem kompletten Dateiname inkl. Path, welcher in der Variable bild_fname gespeichert wird.
      bild_fname = testDir + curFile

      # Das geladene Bild wird in der Variable img gespeichert
      # Das Bild wird durch die Funktion tf.keras.utils.load_img geladen
      img = tf.keras.utils.load_img(
          # Dateiname inkl. Path des zu ladenden Testbildes
          bild_fname,
          # Hier wird der Farbenmodus gewählt, durch die bereits definierte Variable usedColorMode, also entweder grayscale oder rgb, kommt drauf an, was das Modell verlangt
          color_mode=usedColorMode,
          # Die geladenen Bilder werden durch die definierte Variable IMG_SIZE in der Grösse verändert wenn nötig (wenn sie z.B. viel grösser sind), in meinem Fall zu 64x64 Pixel
          target_size=(IMG_SIZE, IMG_SIZE)
      )
      # Das Image in ein Feld (array) umbauen, denn keras arbeitet nur mit diesen arrays.
      # Bei Trainieren des Models ist das von der Function tf.keras.utils.image_dataset_from_directory "miterledigt" worden
      img_array = tf.keras.utils.img_to_array(img)

      # ein array von bildern anlegen, das aber nur 1 Bild enthält. Die folgende model.predict function braucht als Eingabe so ein Array, deshalb dieser Umweg
      img_array = tf.expand_dims(img_array, 0)

      # Berechnet die Prediction für das gerade geladene Bild
      predictions = model.predict(img_array,verbose=0)
      # Beispiel für eine predictions, die in der Zeile vorher berechnet wurde. Man sieht die 18 Bewertungen für die 18 Klassen
      #[[14229 -6334 3480 2634 -784 -6531 -4458 -7535 -6561 630 -2632 -6097 2005 -3240 2218 -4952 -5617
      #  4911]]

      # berechnet den score aus der prediction mit der softmax methode
      score = tf.nn.softmax(predictions[0])
      # Beispiel für den score aus dem prediction Beispiel
      # Man sieht das Klasse 1 den höchsten Wert (14229) hat und natürlich hat im tensor die position 1 eine 1, alle anderen 17 sind 0
      # tf.Tensor([ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0], shape=(18,), dtype=float32)

      # mit numpy wird jetzt die position der 1 bestimmt, ist in unserem Fall die 0, denn das Zählung beginnt ja bei 0 bis 17
      predictedClass = np.argmax(score)

      # Die ersten drei Zahlen des Dateinamens "enthalten" den Klassennamen des Bildes
      # mit der index function wird die Positions dieser 3 Zahlen in der classnames liste gesucht. Z.B. '001' -> 0, '150' -> 17 siehe Anfang dieses Codeblocks
      actualClass = classnames.index(curFile[0:3])

      # "Treffer" in der Tabelle hochzählen um 1
      F1arr[predictedClass][actualClass]+= 1

# Berechnen der Zeilensummen (TP und FP aus der Sicht der Predicted Class, des Ergebnisses der Prediction)
for zeile in range(18):
    # Die Variable summeZeile wird für jede Zeile erstmal auf Null gesetzt.
    summeZeile = 0
    for spalte in range(18):
        # Die Werte jeder Zelle in der Zeile wird zu summeZeile dazuaddiert
        summeZeile += F1arr[zeile][spalte]
    # Nun wird dieses Ergebniss von allen Zellen in der Spalte 18 gespeichert (was eigentlich die 19. Spalte ist, da die Zählung ja bei 0 beginnt)
    F1arr[zeile][18]= summeZeile


# Berechnen der Spaltensummen (TP und FN aus der Sicht der Actual Class, also der Bilder). Hier geht es bis 19, damit auch die Zeilensumme (Spalte 18) von vorher aufsummiert werden
for spalte in range(19):
    # Die Variable summeSpalte wird genullt
    summeSpalte = 0
    for zeile in range(18):
        # Der Wert jeder Zelle in der Spalte wird zu summeSpalte dazuaddiert
        summeSpalte += F1arr[zeile][spalte]
    # Nun wird dieses Ergebniss von allen Zellen in der 18ten Zeile versorgt
    F1arr[18][spalte]= summeSpalte


# Ausgabezeilen für Numpy verlängern für die breite Tabelle und Spalten etwas verkleinern
np.set_printoptions(formatter={'float_kind': '{:2.0f}'.format},linewidth=100)

# Der Inhalt der Variable F1arr wird unten angezeigt, hierbei ist es ein NumPy, welche die Klassifikations Ergebnisse beinhält genauer die Wahrheitsmatrix
print(F1arr)



[[10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10]
 [ 0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10]
 [ 0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10]
 [ 0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10]
 [ 0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0 10]
 [ 0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0 10]
 [ 0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0  0  0  5]
 [ 0  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0  0  5]
 [ 0  0  0  0  0  0  0  0  7  0  0  0  0  0  0  0  0  0  7]
 [ 0  0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  4]
 [ 0  0  0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  4]
 [ 0  0  0  0  0  0  0  0  0  0  0  7  0  0  0  0  0  0  7]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  9  0  0  0  0  0  9]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  0  0  0 10]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  0  0 10]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  0 10]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0

In [9]:
# Summen für die Durchschnittsberechnung auf 0 setzen
sumPrecision=0
sumRecall=0
sumF1=0
sumAccuracy=0

for klasse in range(18):  # Ab hier wurde der Code mit Hilfe von Google Gemini geschrieben
  # F1arr[klasse][klasse] enthält die korrekten Vorhersagen, der jeweiligen Klasse, also TP
  # F1arr[klasse][18] enthält die Gesamtheit der Vorhersagen für diese Klasse, egal ob korrekt oder flasch, also TP+FP
  precision = F1arr[klasse][klasse] / F1arr[klasse][18]
  # F1arr[klasse][klasse] enthält die korrekten Vorhersagen, der jeweiligen Klasse, also TP
  # F1arr[18][klasse] enthält die wirkliche Anzahl von Puzzleteilen, welche zu dieser Klasse gehören, also TP+FN
  recall = F1arr[klasse][klasse] / F1arr[18][klasse]
  # F1 entsteht aus precision und recall
  F1 = 2 * precision * recall / (precision + recall)

  # Die Genauigkeit aller Klassen wird zusammngerechnet
  sumPrecision += precision
  # Der Recall aller Klassen wird zusammngerechnet
  sumRecall += recall
  # Der F1 Wert aller Klassen wird zusammngerechnet
  sumF1 += F1
  # Accuracy kann nur über alle Klassen hinweg gerechnet werden, somit werden hier die Werte der korrekt Vorhergesagten jeder Klasse addiert
  sumAccuracy += F1arr[klasse][klasse]

  # Ausgabe der Einzelwerte pro Klasse
  # die {} sind Platzhalter, welche mit klasse+1 (es gibt keine 0te Klasse, deswegen wird immer eine 1 addiert) ausgefüllt werden
  # der zweite Platzhalter ist jeweils für precision, recall oder F1. :4.2f gibt an, dass der Wert immer mit 4 Zeichen dargestellt wird (Punkt und Nachkommastellen zählen mit) und 2 Dezimalstellen dargestellt wird
  print("Precision für Klasse {}: {:4.2f}".format(klasse+1,precision))
  print("Recall für Klasse    {}: {:4.2f}".format(klasse+1,recall))
  print("F1 für Klasse        {}: {:4.2f}".format(klasse+1,F1))
  print("")

# Ausgabe der durchschnittlichen Werte über alle Klassen
# für den Durchschnitt von Precision, Recall, Accuracy und F1 muss nun noch durch die Anzahl der Klassen gerechnet werden.
# die {} sind Platzhalter, hier wiederrum mit der Einschränkung, dass Wert mit maximal 4 und 2 Dezimalstellen angezeigt wird
print("Durchschnittliche Precision: {:4.2f}".format(sumPrecision/18))
print("Durchschnittliche Recall:    {:4.2f}".format(sumRecall/18))
print("Durchschnittliche Accuracy:  {:4.2f}".format(sumAccuracy/F1arr[18][18]))
print("Durchschnittliche F1:        {:4.2f}".format(sumF1/18))


Precision für Klasse 1: 1.00
Recall für Klasse    1: 1.00
F1 für Klasse        1: 1.00

Precision für Klasse 2: 1.00
Recall für Klasse    2: 1.00
F1 für Klasse        2: 1.00

Precision für Klasse 3: 1.00
Recall für Klasse    3: 1.00
F1 für Klasse        3: 1.00

Precision für Klasse 4: 1.00
Recall für Klasse    4: 1.00
F1 für Klasse        4: 1.00

Precision für Klasse 5: 1.00
Recall für Klasse    5: 1.00
F1 für Klasse        5: 1.00

Precision für Klasse 6: 1.00
Recall für Klasse    6: 1.00
F1 für Klasse        6: 1.00

Precision für Klasse 7: 1.00
Recall für Klasse    7: 1.00
F1 für Klasse        7: 1.00

Precision für Klasse 8: 1.00
Recall für Klasse    8: 1.00
F1 für Klasse        8: 1.00

Precision für Klasse 9: 1.00
Recall für Klasse    9: 1.00
F1 für Klasse        9: 1.00

Precision für Klasse 10: 1.00
Recall für Klasse    10: 1.00
F1 für Klasse        10: 1.00

Precision für Klasse 11: 1.00
Recall für Klasse    11: 1.00
F1 für Klasse        11: 1.00

Precision für Klasse 12: 1